# Splitting by the Number of Beams per Hit

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from tabulate import tabulate
from astropy.time import Time
from tqdm import tqdm
import csv

Ok so to start, I want to start filtering by the signal to noise ratio. In doing this, we make an assumption about the potential power of alien civilizations, so we start with a very high signal to noise ratio to cap at (100).

Now start by opening the pickle file.

In [2]:
with open('../../Pickle_Files/subset_3.pkl', 'rb') as f:

    data = pickle.load(f) # deserialize using load()
    
print(data.shape)

(805064, 24)


In [3]:
snr_filtered = data[data["signal_snr"] < 100]

with open('snr_filtered.pkl', 'wb') as f:  # open a text file
    pickle.dump(snr_filtered, f) # serialize the list
f.close()

Now that we have filtered by the signal to noise ratio, we want to go through and only keep the hits which have a corresponding hit in both the incoherent and coherent beams. So how do I do this? I think it would make sense to set up a similar indexing to what I had in the preliminary filtering. From there, it could make sense to get rid of all of the single beam hits, which would either be a hit in only the incoherent beam, or a hit in one of the coherent beams with nothing in the incoherent beam.

Save that all as another pickle file so that I don't have to keep running these.

In [4]:
with open('snr_filtered.pkl', 'rb') as f:

    snr_data = pickle.load(f) # deserialize using load()
    
print(snr_data.shape)

(772551, 24)


Now Jared has sent some of the preliminary kurtosis information. So I want to remove any of the channels that have a lot of RFI before continuing.

Start by opening the csv file with the frequency binning information

In [5]:
source = '../../Preliminary_Data_Files/prelim_kurt_bins.csv'
#csv.reader(source)

file = open(source)

with open(source, newline='') as f:
    reader = csv.reader(f)
    row1 = next(reader)

data = pd.read_csv(source)

lower_freq_bin = [float(x) for x in data['rfi_freq_bin_bots'] if str(x) != 'nan']
upper_freq_bin = [float(x) for x in data['rfi_freq_bin_tops'] if str(x) != 'nan']

Now find all of the frequencies which are in between the lower and upper edges of the frequency bins. Index them and then remove these indices from the original list of indices so that we are left with just the indices which we would like to keep.

In [ ]:
signal_frequency = np.array(snr_data['signal_frequency']).tolist()

source_name = np.array(snr_data['sourceName'])
unique_source_name = np.unique(source_name)

discarded_frequencies = []
for i in tqdm(range(len(source_name))):
    for j in range(len(lower_freq_bin)):
        if signal_frequency[i] > lower_freq_bin[j] and signal_frequency[i] < upper_freq_bin[j]:
            discarded_frequencies.append(i)

kept_frequencies = list(set(range(len(signal_frequency))) - set(discarded_frequencies))

Now run through and keep just those rows in the table which correspond to the indices which we decided above we would like to keep. We still need to define the data frame that we are appending too, but the rest is pretty easy.

In [ ]:
df1 = pd.DataFrame({"file_path":[],
                    "hit_file_enumeration":[],
                    "signal_frequency":[],
                    "signal_index":[],
                    "signal_driftSteps":[],
                    "signal_driftRate":[],
                    "signal_snr":[],
                    "signal_coarseChannel":[],
                    "signal_numTimesteps":[],
                    "signal_power":[],
                    "signal_incoherentPower":[],
                    "sourceName":[],
                    "fch1":[],
                    "foff":[],
                    "tstart":[],
                    "tsamp":[],
                    "ra":[],
                    "dec":[],
                    "telescopeId":[],
                    "numTimesteps":[],
                    "numChannels":[],
                    "coarseChannel":[],
                    "startChannel":[],
                    "beam":[]})

appending_rows = snr_data.iloc[kept_frequencies]
    
df1 = df1.append(appending_rows, ignore_index = True)

And now save that data frame to a pickle file so that we can open it and work with that information whenever we want to and need to.

In [ ]:
with open('snr_kurtosis.pkl', 'wb') as f:  # open a text file
    pickle.dump(df1, f) # serialize the list
f.close()

In [ ]:
with open('snr_kurtosis.pkl', 'rb') as f:

    snr_data_kurtosis = pickle.load(f) # deserialize using load()
    
print(snr_data_kurtosis.shape)

Comparing where the data in the data frames before and after the kurtosis cut were distributed.

In [ ]:
######### Looking at the number of hits in each field of view #########
file_path = snr_data["file_path"]
unique_file_path = np.unique(file_path) #the array of unique fields of view
print(len(unique_file_path))
print(len(np.unique(snr_data["tstart"])))

file_path_kurtosis = snr_data_kurtosis["file_path"]
unique_file_path_kurtosis = np.unique(file_path) #the array of unique fields of view
print(len(unique_file_path_kurtosis))
print(len(np.unique(snr_data_kurtosis["tstart"])))

Now I would like to split the file up into two smaller .pickle files. This split is based on the number of beams that a hit is present in. Because I want to compare the powers between the incoherent beam and the coherent beams, I want to take out all of the hits which are only present in one beam.

There are two reasons why a hit might be present in only one beam. If the signal is incredibly weak, then it might get averaged out in the incoherent beam, but would still be present in a coherent beam. However, if the signal is not localized to one of the 5 closest stars within a given field of view, then it might only be present in the incoherent beam. Thsi is the case with something like Voyager or if there were aliens on a rogue planet or on a space ship flying around in the ISM.

In [ ]:
######### Start by making a blank data frame that we can append information to #########
df2 = pd.DataFrame({"file_path":[],
                    "hit_file_enumeration":[],
                    "signal_frequency":[],
                    "signal_index":[],
                    "signal_driftSteps":[],
                    "signal_driftRate":[],
                    "signal_snr":[],
                    "signal_coarseChannel":[],
                    "signal_numTimesteps":[],
                    "signal_power":[],
                    "signal_incoherentPower":[],
                    "sourceName":[],
                    "fch1":[],
                    "foff":[],
                    "tstart":[],
                    "tsamp":[],
                    "ra":[],
                    "dec":[],
                    "telescopeId":[],
                    "numTimesteps":[],
                    "numChannels":[],
                    "coarseChannel":[],
                    "startChannel":[],
                    "beam":[]})

######## This for loop should go through and add all of the tables that have the beam anomalies to a new table #########
for i in tqdm(range(len(unique_file_path_kurtosis))):
    fov_subset = snr_data_kurtosis.loc[snr_data_kurtosis['file_path'] == 
                                       unique_file_path_kurtosis[i]].sort_values(by = ["signal_frequency"]) 
    #select each subset
    
    freq = np.array(fov_subset["signal_frequency"]) #define just the column for frequencies
    unique_freq = np.unique(freq) #find the array of unique frequencies
    
    #Here we want to find how many hits each frequency had between the 5 coherent beams and the one incoherent beam
    hits_per_freq = []
    for k in range(len(unique_freq)):
        hits_per_freq.append(np.count_nonzero(np.array(freq) == unique_freq[k]))
   
    #Each frequency should have a maximum of 6 hits if it is widespread RFI
    #indices = np.where(np.array(hits_per_freq) == 1)
    indices = np.where(np.array(hits_per_freq) != 1)
    values_greater_than_1 = np.array(unique_freq)[indices]
    
    new_indices = np.concatenate(np.where(np.isin(np.array(fov_subset.sort_values(by = ["signal_frequency", "beam"])
                                                           ["signal_frequency"]), values_greater_than_1) == True))
    
    appending_rows = fov_subset.iloc[new_indices]
    
    df2 = df2.append(appending_rows, ignore_index = True)
    
with open('greater_than_1_kurtosis.pkl', 'wb') as f:  # open a text file
    pickle.dump(df2, f) # serialize the list
f.close()
print(df1.shape)

Ok now index everything again, but this time, try to figure out how to make it into subarrays or a list of smaller arrays. Then see if there is a way to only keep the sets of arrays that only contain a 5 (incoherent beam). But we'll start this with just the hits that are present in 4 and 5 beams. Because I have already removed the hits that are present in all 6 beams, these are the most. It is most likely low intensity RFI if it is present in 4-5 beams but no incoherent beam, but this will be checked by examining the SNR and the total power.

The reason I am not looking at the hits that are present in 2-3 beams first is because there are more reasons as to why these might not have a corresponding incoherent beam. If the two unique sources are very close to each other to the point where they would not be fully resolved, then we would see the same hit occurring in multiple beams. So once we have these split up, I'll look at how far apart the beams are from each other to test this idea.

In [ ]:
with open('greater_than_1_kurtosis.pkl', 'rb') as f:

    greater_than_1_kurtosis = pickle.load(f) # deserialize using load()
    print(greater_than_1_kurtosis.shape)
    
with open('1_hit_kurtosis.pkl', 'rb') as f:

    data_1_kurtosis = pickle.load(f) # deserialize using load()
    print(data_1_kurtosis.shape)

In [ ]:
######### Looking at the number of hits in each field of view #########
file_path = greater_than_1_kurtosis["file_path"]
unique_file_path = np.unique(file_path) #the array of unique fields of view
print(greater_than_1_data.shape)
print(len(unique_file_path))
print(len(np.unique(greater_than_1_data["tstart"])))

file_path_1 = data_1_kurtosis["file_path"]
unique_file_path_1 = np.unique(file_path_1) #the array of unique fields of view
print(data_1_kurtosis.shape)
print(len(unique_file_path_1))
print(len(np.unique(data_1["tstart"])))

This next section is looking to map the observations we are currently left with. I use seaborn to quickly show the frequencies and the number of hits per source or per field of view in each of these plots below.

In [ ]:
source_name = np.unique(greater_than_1_kurtosis["sourceName"])
source = greater_than_1_kurtosis["sourceName"]

hits_per_source = []
for i in tqdm(range(len(source_name))):
    hits_per_source.append(np.count_nonzero(source == source_name[i]))

In [ ]:
indices = [0]
for i in range(len(hits_per_source)-1):
    indices.append(indices[i] + hits_per_source[i])
    
print(len(indices))

In [ ]:
hits_info = pd.DataFrame({'Source': np.array(greater_than_1_kurtosis.sort_values(by = ["sourceName", "file_path"])["sourceName"])[indices], 
        'Number of Hits': np.array(hits_per_source), 
        'Right Ascension': np.array(greater_than_1_kurtosis.sort_values(by = ["sourceName", "file_path"])["ra"])[indices], 
       'Declination': np.array(greater_than_1_kurtosis.sort_values(by = ["sourceName", "file_path"])["dec"])[indices]})

display(hits_info)

with open('hits_per_source_2.pkl', 'wb') as f:  # open a text file
    pickle.dump(hits_info, f) # serialize the list
f.close()

In [ ]:
with open('hits_per_source_2.pkl', 'rb') as f:
    hits_info = pickle.load(f)

In [ ]:
sns.set_theme()
sns.relplot(data=hits_info[0:-1], x=hits_info["Right Ascension"][0:-1], y=hits_info["Declination"][0:-1], 
            hue=hits_info["Number of Hits"][0:-1], size = hits_info["Number of Hits"][0:-1])

In [ ]:
info = pd.DataFrame({'Source': np.array(greater_than_1_kurtosis.sort_values(by = ["sourceName", "file_path"])["sourceName"]), 
        'File Path': np.array(greater_than_1_kurtosis.sort_values(by = ["sourceName", "file_path"])["file_path"]), 
        'Right Ascension': np.array(greater_than_1_kurtosis.sort_values(by = ["sourceName", "file_path"])["ra"]), 
       'Declination': np.array(greater_than_1_kurtosis.sort_values(by = ["sourceName", "file_path"])["dec"])})

display(info)

In [ ]:
def countFreq(arr, n):
    
    # Mark all array elements as not visited
    visited = [False for i in range(n)]
    fp_list = []
    # Traverse through array elements and count frequencies
    for i in range(n):
         
        # Skip this element if already processed
        if (visited[i] == True):
            continue
 
        # Count frequency
        count = 1
        for j in range(i + 1, n, 1):
            if (arr[i] == arr[j]):
                visited[j] = True
                count += 1
         
        #print(count)
        fp_list.append(count)
    return len(fp_list)
        
#Driver Code
arr = np.array(info["File Path"][693038:-1])
n = len(arr)
countFreq(arr, n)

In [ ]:
fovs_per_source = []
for i in range(len(indices)-1):
    arr = np.array(info["File Path"][indices[i]:indices[i+1]])
    n = len(arr)
    fovs_per_source.append(countFreq(arr, n))
#print(len(fovs_per_source))
fovs_per_source.append(18194)

In [ ]:
fovs_info = pd.DataFrame({'Source': np.array(greater_than_1_kurtosis.sort_values(by = ["sourceName", "file_path"])["sourceName"])[indices], 
        'Number of FOVs': np.array(fovs_per_source), 
        'Right Ascension': np.array(greater_than_1_kurtosis.sort_values(by = ["sourceName", "file_path"])["ra"])[indices], 
       'Declination': np.array(greater_than_1_kurtosis.sort_values(by = ["sourceName", "file_path"])["dec"])[indices]})

display(fovs_info)

with open('fovs_per_source_2.pkl', 'wb') as f:  # open a text file
    pickle.dump(fovs_info, f) # serialize the list
f.close()

In [ ]:
with open('fovs_per_source_2.pkl', 'rb') as f:
    fovs_info = pickle.load(f)

In [ ]:
sns.set_theme()
sns.relplot(data=fovs_info[0:-1], x=fovs_info["Right Ascension"][0:-1], y=fovs_info["Declination"][0:-1], 
            hue=fovs_info["Number of FOVs"][0:-1], size = fovs_info["Number of FOVs"][0:-1], palette = "ch:s=.25,rot=-.25")

Here I am trying to look at which beams are in each of the hits which have multiple beams. That doesn't make much sense. So... Because each possibly interesting hit should have at least one hit in the coherent beam and the same hit in the incoherent beam, we want to make sure that we are only keeping those hits which present themselves in both the incoherent beam and a coherent beam. So here we are going to put each of the sets of hits into smaller array that contain the beam numbers for the corresponding things idk

In [ ]:
with open('greater_than_1_kurtosis.pkl', 'rb') as f:
    greater_than_1_data = pickle.load(f)

In [ ]:
file_path = greater_than_1_data["file_path"]
unique_file_path = np.unique(file_path) #the array of unique fields of view

print(len(unique_file_path), len(file_path))

In [ ]:
array = []
array_indexing = []
for i in tqdm(range(len(unique_file_path))):
    data_fov_subset = greater_than_1_data.loc[greater_than_1_data['file_path'] == unique_file_path[i]] #select each subset    
    data_fov_subset_sort = data_fov_subset.sort_values(by = ["signal_frequency"], ascending = True)
    
    freq = np.array(data_fov_subset["signal_frequency"]) #define just the column for frequencies
    #print(len(freq))
    beam = np.array(data_fov_subset["beam"])
    freq_list = freq.tolist()
    unique_freq = np.unique(freq) #find the array of unique frequencies
    
    #Here we want to find how many hits each frequency had between the 5 coherent beams and the one incoherent beam
    hits_per_freq = []
    for j in range(len(unique_freq)):
        hits_per_freq.append(np.count_nonzero(freq == unique_freq[j]))
        
        this_freq = unique_freq[j]
    
        frequency_indices = []
        index = []
        for k in range(len(freq)):
            if freq[k] == unique_freq[j]:
                frequency_indices.append(beam[k])
                index.append(i)

        #print(frequency_indices)
        array.append(frequency_indices)
        array_indexing.append(index)
print(len(array))

In [ ]:
array_length = []
for i in range(len(array)):
    array_length.append(len(array[i]))

weirdos = []
for i in range(len(array_length)):
    if array_length[i] == 5:
        if 5.0 not in array[i]:
            #print(array[i])
            weirdos.append(i)
#print(weirdos) #This tells the indexing of which of the smaller arrays have the strange behavior
#I am characterizing the weirdos as being the smaller arrays which I would expect to have an incoherent beam, but for whatever 
#reason do not
print(len(weirdos))

#Now these next two arrays are looking at which fields of view the "weirdos" are present in (numbers) and how many weirdos are
#in each field of view (counting)
numbers = []
for i in range(len(weirdos)):
    numbers.append(array_indexing[weirdos[i]][0])

counting = []
for i in range(len(np.unique(numbers))):
    counting.append(np.count_nonzero(numbers == np.unique(numbers)[i]))
#print(counting)

#print(np.max(counting))
#print(np.count_nonzero(np.array(counting) == 1))

In [ ]:
five = []
for i in range(len(array_length)):
    if array_length[i] == 2:
        five.append(i)
print(len(five))

I think it'll be easier to not make a mistake if I further break this up into subsets based on the number of beams per hit. And then from there, I should be able to more easily flag those which are lacking an incoherent beam.

In [ ]:
######### Start by making a blank data frame that we can append information to #########
df2 = pd.DataFrame({"file_path":[],
                    "hit_file_enumeration":[],
                    "signal_frequency":[],
                    "signal_index":[],
                    "signal_driftSteps":[],
                    "signal_driftRate":[],
                    "signal_snr":[],
                    "signal_coarseChannel":[],
                    "signal_numTimesteps":[],
                    "signal_power":[],
                    "signal_incoherentPower":[],
                    "sourceName":[],
                    "fch1":[],
                    "foff":[],
                    "tstart":[],
                    "tsamp":[],
                    "ra":[],
                    "dec":[],
                    "telescopeId":[],
                    "numTimesteps":[],
                    "numChannels":[],
                    "coarseChannel":[],
                    "startChannel":[],
                    "beam":[]})

######## This for loop should go through and add all of the tables that have the beam anomalies to a new table #########
for i in tqdm(range(len(unique_file_path))):
    fov_subset = greater_than_1_data.loc[greater_than_1_data['file_path'] == 
                                       unique_file_path[i]].sort_values(by = ["signal_frequency"]) 
    #select each subset
    
    freq = np.array(fov_subset["signal_frequency"]) #define just the column for frequencies
    unique_freq = np.unique(freq) #find the array of unique frequencies
    
    #Here we want to find how many hits each frequency had between the 5 coherent beams and the one incoherent beam
    hits_per_freq = []
    for k in range(len(unique_freq)):
        hits_per_freq.append(np.count_nonzero(np.array(freq) == unique_freq[k]))
   
    #Change this value here to set it up for different numbers of beams that a hit is present in here
    indices = np.where(np.array(hits_per_freq) == 2) 

    values_greater_than_1 = np.array(unique_freq)[indices]
    
    new_indices = np.concatenate(np.where(np.isin(np.array(fov_subset.sort_values(by = ["signal_frequency", "beam"])
                                                           ["signal_frequency"]), values_greater_than_1) == True))
    
    appending_rows = fov_subset.iloc[new_indices]
    
    df2 = df2.append(appending_rows, ignore_index = True)
    
with open('2_beams_per_hit.pkl', 'wb') as f:  # open a text file
    pickle.dump(df2, f) # serialize the list
f.close()
print(df2.shape)

Now let's open up the .pickle file that has all of the hits that were present in five beams and run the same code as above to figure out which of them are lacking an incoherent beam. Above I called these the weirdos. It should then be easier to index these and then filter all of them out.

My plan right now is to clump them together based on the hit. This would be based on the field of view, then based on the frequency. Then within each of those clumps, look at the array of the beam numbers and if beam 5 (the incoherent beam) is present, then the entire clump gets appended to the data frame. But I want to only do this for the hits that are present in 4 or 5 beams.

In [ ]:
with open('2_beams_per_hit.pkl', 'rb') as f:
    hit_2_data = pickle.load(f)
print(hit_2_data.shape)
file_path_2 = hit_2_data["file_path"]
unique_file_path_2 = np.unique(file_path_2) #the array of unique fields of view
    
with open('3_beams_per_hit.pkl', 'rb') as f:
    hit_3_data = pickle.load(f)
print(hit_3_data.shape)
file_path_3 = hit_3_data["file_path"]
unique_file_path_3 = np.unique(file_path_3) #the array of unique fields of view

with open('4_beams_per_hit.pkl', 'rb') as f:
    hit_4_data = pickle.load(f)
print(hit_4_data.shape)
file_path_4 = hit_4_data["file_path"]
unique_file_path_4 = np.unique(file_path_4) #the array of unique fields of view

with open('5_beams_per_hit.pkl', 'rb') as f:
    hit_5_data = pickle.load(f)
print(hit_5_data.shape)
file_path_5 = hit_5_data["file_path"]
unique_file_path_5 = np.unique(file_path_5) #the array of unique fields of view

In [ ]:
df2 = pd.DataFrame({"file_path":[],
                    "hit_file_enumeration":[],
                    "signal_frequency":[],
                    "signal_index":[],
                    "signal_driftSteps":[],
                    "signal_driftRate":[],
                    "signal_snr":[],
                    "signal_coarseChannel":[],
                    "signal_numTimesteps":[],
                    "signal_power":[],
                    "signal_incoherentPower":[],
                    "sourceName":[],
                    "fch1":[],
                    "foff":[],
                    "tstart":[],
                    "tsamp":[],
                    "ra":[],
                    "dec":[],
                    "telescopeId":[],
                    "numTimesteps":[],
                    "numChannels":[],
                    "coarseChannel":[],
                    "startChannel":[],
                    "beam":[]})

for i in tqdm(range(int(len(file_path_2)/2))):
#for i in range(4):
    data_fov_subset = hit_5_data.loc[i*2:(i*2)+1] #select each subset    
    #It is already organized by frequency so we don't need to sort by frequency

    #display(data_fov_subset)
    freq = np.array(data_fov_subset["signal_frequency"]) #define just the column for frequencies
    beam = np.array(data_fov_subset["beam"])
    #print(len(beam))
    freq_list = freq.tolist()
    unique_freq = np.unique(freq) #find the array of unique frequencies
    
    if 5.0 not in beam:
        #print('yay')
        appending_rows = data_fov_subset
        df2 = df2.append(appending_rows, ignore_index = True)
        #print(appending_rows.shape)
    else:
        continue
    
with open('2_beams_coherent.pkl', 'wb') as f:  # open a text file
    pickle.dump(df2, f) # serialize the list
f.close()
print(df2.shape)

In [ ]:
with open('4_beams.pkl', 'rb') as f:
    hit_4 = pickle.load(f)
print(hit_4.shape)
file_path_4 = hit_4["file_path"]
unique_file_path_4 = np.unique(file_path_4) #the array of unique fields of view

with open('5_beams.pkl', 'rb') as f:
    hit_5 = pickle.load(f)
print(hit_5.shape)
file_path_5 = hit_5["file_path"]
unique_file_path_5 = np.unique(file_path_5) #the array of unique fields of view

Now divide the incoherent power by each of the coherent powers of the coherent beams. This should be close to 13 so see if we can print all of these values out. Between all of this, there should be a good way of starting to see some candidates. We can then combine this with the information from the drift rates to get an even better idea of what is going on here. This will be done in other Jupyter Notebooks, which can all be found in this same GitHub Repository.